# TTstar benchmarks

In [ ]:
# First, import our libraries...

import pandas as pd
import numpy as np
import plotnine as gg
import sklearn.linear_model
import statsmodels.formula.api as smf
from plotnine import *

In [ ]:
# ...and then load the data files

#xs = pd.read_csv('benchmark-1490607733.csv')
xs = pd.read_csv('benchmark-1491380156.csv')
#xs = pd.read_csv('benchmark-1493248450.csv')
#xs = pd.read_csv('benchmark-1493251003.csv')

# this builds the column `config` as $STAGE-$FEATURES,
# where each feature is the first letter of features enabled,
# Example: ttstar-ivn

xs['config'] = xs['stage'] + '-'
for col in 'inference specialisation verification normalisation compilation'.split():
    # the comparison to `True` is a hack to set NaNs to False
    xs.loc[xs[col] == True, 'config'] += col[0]

# Add a milliseconds column
xs['runtime_ms'] = 1000 * xs['runtime']

# Show the structure
xs.head()

## How much time does erasure analysis take?

The plot and the linear regression below give an answer:
* typechecking alone takes about 2.5 milliseconds
* erasure inference takes about 109 milliseconds

In [ ]:
# ttstar
ttstar = xs.loc[xs['stage'] == 'ttstar']
plot = ggplot(ttstar, aes(x='config', y='runtime_ms')) \
    + geom_boxplot() \
    + geom_hline(yintercept=0, color='white') \
    + coord_flip() \
    + xlab('') + ylab('program runtime [ms]') \
    + ggtitle("TTstar runtime")
    
print(plot)
plot.save('ttstar-runtime.pdf', width=5.6, height=2.6, units='in')

In [ ]:
model = sklearn.linear_model.LinearRegression()
model.fit(
    X=ttstar.loc[:, ('inference', 'verification', 'normalisation')],
    y=ttstar['runtime_ms']
)
print('intercept =', model.intercept_)
print('coefficients =', model.coef_)

In [ ]:
smf.ols('runtime_ms ~ inference + verification + normalisation', data=ttstar).fit().summary()

## How much codegen time does erasure analysis save?

The linear regression and the plot below give reassuring results:
* erasure inference shaves about 2000 milliseconds off codegen time
* typechecking has no statistically significant effect on codegen time

In [ ]:
csc = xs.loc[xs['stage'] == 'csc']

smf.ols('runtime_ms ~ inference + verification + normalisation', data=csc).fit().summary()

In [ ]:
plot = ggplot(csc, aes(x='config', y='runtime_ms')) \
    + geom_boxplot() \
    + geom_hline(yintercept=0, color='white') \
    + xlab('') + ylab('code generation time [ms]') \
    + coord_flip() \
    + ggtitle("Code generation")
    
print(plot)
plot.save('csc-runtime.pdf', width=5.6, height=2.6, units='in')

## Runtime effect of erasure

In [ ]:
exec = xs.loc[(xs['stage'] == 'execution') & ~xs['normalisation'] & ~xs['verification']]

ggplot(exec.loc[exec['inference']], aes(x='input_size', y='runtime', color='config')) \
    + geom_point(alpha=0.3) \
    + ggtitle('Effect of compilation on erased programs')

In [ ]:
ggplot(exec.loc[exec['compilation'] == False],
       aes(x='input_size', y='runtime_ms', color='config')) \
    + geom_point(alpha=0.3) \
    + facet_grid('. ~ inference', scales='free_x', labeller='label_both') \
    + xlab('input size') + ylab('runtime (s)') \
    + guides(color=False) # remove legend for colour
    #ggtitle("Effect of erasure on interpreted programs")
#ggsave('exec-runtime-interpreted-erasure.pdf', width=5.6, height=3.0, units='in')

In [ ]:
ggplot(exec.loc[exec['compilation'] == True],
       aes(x='input_size', y='runtime_ms', color='config')) \
    + geom_point(alpha=0.3) \
    + facet_grid('. ~ inference', scales='free_x', labeller='label_both') \
    + xlab('input size') + ylab('runtime (s)') \
    + guides(color=False) # remove legend for colour

#ggtitle("Effect of erasure on interpreted programs")
#ggsave('exec-runtime-interpreted-erasure.pdf', width=5.6, height=3.0, units='in')

In [ ]:
ggplot(
    exec.loc[
        exec['compilation']
        & ~exec['inference']
        & (exec['input_size'] >= 32)
    ],
    aes(
        x='np.log(runtime)',
        y='np.log(input_size)')
) \
    + geom_point()

In [ ]:
smf.ols(
    'np.log(runtime) ~ np.log(input_size)',
    data=exec.loc[exec['compilation'] & ~exec['inference'] & (exec['input_size'] >= 32)],
).fit().summary()

In [ ]:
ggplot(
    exec.loc[
        exec['compilation']
        & exec['inference']
        & (exec['input_size'] >= 32)
    ],
    aes(
        x='np.log(runtime)',
        y='np.log(input_size)')
) \
    + geom_point()

In [ ]:
smf.ols(
    'np.log(runtime) ~ np.log(input_size)',
    data=exec.loc[exec['compilation'] & exec['inference']],
).fit().summary()